02 — Baseline Text Model (LabelCraft 2025)
Этот ноутбук строит простой текстовый baseline:

* использует labeled_train.parquet и category_tree.csv;

* готовит поле text_clean;

* обучает TF-IDF + Logistic Regression;

* считает macro/micro F1 на валидации как ориентир для будущих моделей.

# Загрузка данных

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# папка, куда хотим положить код
target_dir = "/content/drive/MyDrive/labelcraft-2025-ml-challenge"

# если там уже есть старые файлы, эту строку лучше не трогать;
# на первом запуске папка, скорее всего, пуста
print("Содержимое перед клоном:", os.listdir(os.path.dirname(target_dir)))

# переходим в MyDrive
%cd /content/drive/MyDrive

# если репозитория ещё нет — клонируем
if not os.path.exists("labelcraft-2025-ml-challenge"):
    !git clone https://github.com/desve/labelcraft-2025-ml-challenge.git
else:
    print("Репозиторий уже существует, клонировать не будем.")

Содержимое перед клоном: ['AML-54', 'the-nature-conservancy-fisheries-monitoring', 'Colab Notebooks', 'Deep Learning', 'VK', 'MTS', 'history_exploration_2.ipynb', 'VK_Project', 'VK_Projecthistory_grouped.pkl', 'VK_Projectad_x.pt', 'VK_Project_v2', 'vk-text-to-image-demo', 'VK_Project_v3', 'LabelCraft_2025_private', 'LabelCraft_2025', 'labelcraft-2025-ml-challenge', 'experiments']
/content/drive/MyDrive
Репозиторий уже существует, клонировать не будем.


In [3]:
import sys

project_root = "/content/drive/MyDrive/labelcraft-2025-ml-challenge"

if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root:", project_root)
print("src in dir:", "src" in os.listdir(project_root))

Project root: /content/drive/MyDrive/labelcraft-2025-ml-challenge
src in dir: True


In [4]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 50)

print("Setup OK")

DATA_DIR = "/content/drive/MyDrive/LabelCraft_2025/data"

train_path = os.path.join(DATA_DIR, "labeled_train.parquet")
categories_path = os.path.join(DATA_DIR, "category_tree.csv")

print("DATA_DIR:", DATA_DIR)
print("train_path:", train_path)
print("categories_path:", categories_path)

Setup OK
DATA_DIR: /content/drive/MyDrive/LabelCraft_2025/data
train_path: /content/drive/MyDrive/LabelCraft_2025/data/labeled_train.parquet
categories_path: /content/drive/MyDrive/LabelCraft_2025/data/category_tree.csv


In [5]:
os.makedirs(DATA_DIR, exist_ok=True)
print("Files in DATA_DIR:", os.listdir(DATA_DIR))

train = pd.read_parquet(train_path)
cat_tree = pd.read_csv(categories_path)

print("train:", train.shape)
print("category_tree:", cat_tree.shape)

print("train columns:", train.columns.tolist())
print("category_tree columns:", cat_tree.columns.tolist())

Files in DATA_DIR: ['labeled_train.parquet', 'unlabeled_train.parquet', 'unlabeled_special_prize.parquet', 'category_tree.csv']
train: (716552, 4)
category_tree: (1896, 3)
train columns: ['hash_id', 'source_name', 'attributes', 'cat_id']
category_tree columns: ['cat_id', 'parent_id', 'cat_name']


# Подготовка text и text_clean

Склейка текста: source_name + attributes

In [6]:
train["source_name"] = train["source_name"].fillna("")
train["attributes"] = train["attributes"].fillna("")
train["text"] = train["source_name"] + " " + train["attributes"]

Обрезаем до N слов для baseline

In [7]:
MAX_WORDS = 128

def truncate_text(s, max_words=MAX_WORDS):
    if not isinstance(s, str):
        return ""
    words = s.split()
    if len(words) <= max_words:
        return s
    return " ".join(words[:max_words])

train["text_clean"] = train["text"].apply(truncate_text)

Присоединяем имена категорий (пригодится для анализа, хотя baseline использует только cat_id)

In [8]:
train = train.merge(cat_tree[["cat_id", "cat_name"]], on="cat_id", how="left")

print("Примеры (cat_id, cat_name, text_clean):")
print(train[["cat_id", "cat_name", "text_clean"]].head(5))

Примеры (cat_id, cat_name, text_clean):
   cat_id                              cat_name  \
0   10501       Аксессуары для стиральных машин   
1     140                             Умный дом   
2    1397            Духовые шкафы встраиваемые   
3    3645                     Коврики для мышек   
4   10421  Кабели, разъемы для ПК и электроники   

                                                                                                                                                                                                text_clean  
0  Бойник барабана для стиральной машины Candy, Vestel, Bompani, Whirlpool [{""attribute_id"":8,""attribute_name"":""Поставщик"",""attribute_value"":""Нет бренда""},{""attribute_id"":14,""attribute_n...  
1  Приемное устройство М1 TDM Уютный дом в монтажную коробку для беспроводного управления нагрузкой 2300Вт дальность 30м SQ1508-0213 [{""attribute_id"":8,""attribute_name"":""Поставщик"",""attribute_...  
2  Духовой шкаф электрический Darina 1

# Baseline TF-IDF + Logistic Regression (ужатый под Colab)

In [9]:
SAMPLE_SIZE = 30_000 # можно уменьшить до 15_000 при проблемах с памятью

Подвыборка

In [10]:
sample = train.sample(n=SAMPLE_SIZE, random_state=42)

Убираем классы, которые в sample встретились только 1 раз

In [11]:
counts = sample["cat_id"].value_counts()
valid_cats = counts[counts >= 2].index
sample = sample[sample["cat_id"].isin(valid_cats)].copy()

Гарантируем, что текст — строки без NaN

In [12]:
sample["text_clean"] = sample["text_clean"].fillna("").astype(str)

X_text = sample["text_clean"].values
y = sample["cat_id"].values

print("Размер sample после фильтрации редких классов:", sample.shape)
print("Минимальная частота класса:", sample["cat_id"].value_counts().min())

X_train, X_valid, y_train, y_valid = train_test_split(
X_text, y, test_size=0.2, random_state=42, stratify=y
)

vectorizer = TfidfVectorizer(
max_features=50_000,
ngram_range=(1, 1),
min_df=5
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_valid_tfidf = vectorizer.transform(X_valid)

clf = LogisticRegression(
max_iter=150,
n_jobs=-1,
verbose=1,
multi_class="multinomial"
)

clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_valid_tfidf)

macro_f1 = f1_score(y_valid, y_pred, average="macro")
micro_f1 = f1_score(y_valid, y_pred, average="micro")

print("Baseline TF-IDF + LogisticRegression:")
print(" macro F1:", macro_f1)
print(" micro F1:", micro_f1)

Размер sample после фильтрации редких классов: (29942, 7)
Минимальная частота класса: 2


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Baseline TF-IDF + LogisticRegression:
 macro F1: 0.3730081345382464
 micro F1: 0.8295207881115378


# baseline-модель: TF-IDF (max_features=50k) + Logistic Regression (multinomial);

* обучена на подвыборке размером 30k с фильтрацией редких классов (минимум 2 объекта на класс);

* полученные метрики: macro F1 ≈ 0.37, micro F1 ≈ 0.83.

# Импорт и инициализация менеджера

In [16]:
from src.experiment_manager import ExperimentManager

# создаём (или переиспользуем) менеджер
exp_manager = ExperimentManager(root_dir="experiments")

# описываем конфиг конкретно для этого запуска TF-IDF + LR
config = {
    "task": "labelcraft_2025",
    "dataset": {
        "train_path": DATA_DIR,  # подставь свою переменную с путём к parquet
        "target_col": "cat_id",
        "text_col": "text_clean",
        "max_words": MAX_WORDS,   # если используешь такую переменную
    },
    "cv": {
        "n_splits": 5,
        "stratified": True,
        "random_state": 42,
        "test_size": 0.2,
    },
    "model": {
        "name": "tfidf_logreg",
        "tfidf": {
            "max_features": 50000,
            "ngram_range": (1, 2),
        },
        "logreg": {
            "multi_class": "multinomial",
            "solver": "lbfgs",
            "C": 1.0,
            "max_iter": 100,
            "n_jobs": -1,
        },
    },
    "seed": 42,
    "train_size": len(X_train),
    "valid_size": len(X_valid),
}

experiment_id = exp_manager.register_experiment(
    name="baseline_tfidf_logreg",
    description="Baseline TF-IDF + LogisticRegression на Label Craft",
    config=config,
)

print("Registered experiment_id:", experiment_id)

Registered experiment_id: 9ee27a22-b007-4cf1-8524-eb3c3dd67a14


/content/drive/MyDrive/labelcraft-2025-ml-challenge/src/experiment_manager.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


# Логирование метрик

In [17]:
metrics = {
    "macro_f1": macro_f1,
    "micro_f1": micro_f1,
}

exp_manager.log_metrics(experiment_id=experiment_id, metrics=metrics)

print("Metrics logged for experiment_id:", experiment_id)

Metrics logged for experiment_id: 9ee27a22-b007-4cf1-8524-eb3c3dd67a14


# Просмотр сводки

In [18]:
from src.experiment_manager import ExperimentManager

exp_manager = ExperimentManager(root_dir="experiments")
summary = exp_manager.get_summary()
summary.tail()

,experiment_id,name,description,created_at,config_path,metrics_path,macro_f1,micro_f1
0,95ffff18-521c-4827-b8e0-e93084889cbf,baseline_tfidf_logreg,Baseline TF-IDF + LogisticRegression на Label Craft,2025-12-02T06:58:21,experiments/95ffff18-521c-4827-b8e0-e93084889cbf/config.json,experiments/95ffff18-521c-4827-b8e0-e93084889cbf/metrics.json,NaN,NaN
1,694702ce-32fd-41ef-8d3d-e37cb46aacee,baseline_tfidf_logreg,Baseline TF-IDF + LogisticRegression на Label Craft,2025-12-02T07:01:44,experiments/694702ce-32fd-41ef-8d3d-e37cb46aacee/config.json,experiments/694702ce-32fd-41ef-8d3d-e37cb46aacee/metrics.json,0.373008,0.829521
2,9ee27a22-b007-4cf1-8524-eb3c3dd67a14,baseline_tfidf_logreg,Baseline TF-IDF + LogisticRegression на Label Craft,2025-12-02T07:06:03,experiments/9ee27a22-b007-4cf1-8524-eb3c3dd67a14/config.json,experiments/9ee27a22-b007-4cf1-8524-eb3c3dd67a14/metrics.json,0.373008,0.829521
